In [2]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ge

In [38]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/nyc_air_bnb.csv");
df.show(5)

+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca...|   2845|   Jennifer|          Manhatt

In [39]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [37]:
df=df.withColumn("price",df['price'].cast('int')).\
withColumn("minimum_nights",df['minimum_nights'].cast('int')).\
withColumn("number_of_reviews",df['number_of_reviews'].cast('int')).\
withColumn("reviews_per_month",df['reviews_per_month'].cast('int')).\
withColumn("calculated_host_listings_count",df['calculated_host_listings_count'].cast('int'));

In [9]:
df.count()

49079

# **How many total reviews does each host have?**

In [11]:
df.groupBy('host_id').agg(sum('number_of_reviews')).show(10)

+-------+----------------------+
|host_id|sum(number_of_reviews)|
+-------+----------------------+
| 716306|                   197|
|1203500|                    35|
| 368528|                     1|
|1577493|                    16|
|1390555|                    50|
|1317588|                     3|
|2472680|                   219|
|2155832|                   266|
|2426404|                     6|
|2740824|                    22|
+-------+----------------------+
only showing top 10 rows



# **Show the min & max values of all the numerical variables of the dataset**

In [12]:
ms=df.columns;
input_columns=[];
for i in ms:
  if str(df.schema[i].dataType) == 'IntegerType':
    input_columns.append(i);

In [15]:
input_columns

['price',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [14]:
df.select(input_columns).summary("min","max").show()

+-------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
|summary|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+-------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
|    min|  -74|             0|                0|                0|                             0|               0|
|    max|10000|          1250|              629|               58|                           365|             365|
+-------+-----+--------------+-----------------+-----------------+------------------------------+----------------+



# **Which host has heighest number of reviews**

In [29]:
df.groupBy('host_id').agg(sum('number_of_reviews').alias('Total_number_of_Reviews')).orderBy('Total_number_of_Reviews',ascending=False).show()

+---------+-----------------------+
|  host_id|Total_number_of_Reviews|
+---------+-----------------------+
| 37312959|                   2273|
|   344035|                   2205|
| 26432133|                   2017|
| 35524316|                   1971|
| 40176101|                   1818|
|  4734398|                   1798|
| 16677326|                   1355|
|  6885157|                   1346|
|219517861|                   1281|
| 23591164|                   1269|
| 59529529|                   1229|
| 47621202|                   1205|
| 22959695|                   1157|
| 58391491|                   1154|
| 21641206|                   1062|
|   137814|                   1059|
|156948703|                   1052|
|156684502|                   1046|
|  3441272|                   1013|
|  7831209|                    970|
+---------+-----------------------+
only showing top 20 rows



# **How many nights did most hosts specify for a minimum?**

In [33]:
df.select(avg('minimum_nights')).show()

+-------------------+
|avg(minimum_nights)|
+-------------------+
| 7.1286126280910596|
+-------------------+



# **What is the most expensive neighborhood to stay on average?**

In [35]:
df.groupBy('neighbourhood').agg(avg('price').alias('Average Price')).orderBy('Average Price',ascending=False).show(5)

+--------------+------------------+
| neighbourhood|     Average Price|
+--------------+------------------+
|Fort Wadsworth|             800.0|
|       Woodrow|             700.0|
|      Sea Gate| 548.3333333333334|
|       Tribeca|  490.638418079096|
|     Riverdale|442.09090909090907|
+--------------+------------------+
only showing top 5 rows

